In [ ]:
!pip install tweepy

In [4]:
!pip install vadersentiment

    100% |████████████████████████████████| 133kB 3.7MB/s ta 0:00:01
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
# Dependencies
import tweepy
import numpy as np
import pandas as pd

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [7]:
# Target Search Term
target_terms = ("@SouthwestAir", "@AmericanAir", "@SpiritAirlines",
                "@Virginatlantic", "@Delta", "@AlaskaAir", "@KLM", "@FlyFrontier")

# "Real Person" Filters
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"

# List to hold sentiment
results_list = []

# Loop through all target users
for target in target_terms:

    # Variable for holding the oldest tweet
    oldest_tweet = None

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 10 times
    for x in range(10):

        # Run search around each tweet
        public_tweets = api.search(target, 
                               count=100, 
                               result_type="recent", 
                               max_id=oldest_tweet)

        # Loop through all tweets
        for tweet in public_tweets["statuses"]: #only gets the keys that are nested within statuses.

            # Use filters to check if user meets conditions
            if (tweet["user"]["statuses_count"] > min_tweets and
                tweet["user"]["statuses_count"] < max_tweets and
                tweet["user"]["friends_count"] < max_following and
                tweet["user"]["followers_count"] < max_followers and
                tweet["user"]["lang"] == lang):

                # Run Vader Analysis on each tweet
                    results = analyzer.polarity_scores(tweet["text"])

                # Add each value to the appropriate list
                    compound_list.append(results["compound"])
                    positive_list.append(results["pos"])
                    negative_list.append(results["neg"])
                    neutral_list.append(results["neu"])
                
            # Set the new oldest_tweet value
            oldest_tweet = tweet["id"] - 1
            oldest_tweeter = tweet["created_at"]

    # Create a dictionary of the Average Sentiments
    airline_results = {"Airline":target,
                      "Compound Score":np.mean(compound_list),
                      "Pos Score":np.mean(positive_list),
                      "Neg Score":np.mean(negative_list),
                      "Neut Score":np.mean(neutral_list),
                      "Oldest Tweet":oldest_tweeter}

    # Print the Sentiments
    print(airline_results)
    print("-----")
    
    # Append the dictionary to results_list
    results_list.append(airline_results)

{'Airline': '@SouthwestAir', 'Compound Score': 0.1840533123028391, 'Pos Score': 0.1322018927444795, 'Neg Score': 0.046757097791798104, 'Neut Score': 0.8210362776025237, 'Oldest Tweet': 'Mon Oct 22 14:56:28 +0000 2018'}
-----
{'Airline': '@AmericanAir', 'Compound Score': -0.002517535545023697, 'Pos Score': 0.08508214849921011, 'Neg Score': 0.08348815165876777, 'Neut Score': 0.831436018957346, 'Oldest Tweet': 'Mon Oct 22 19:15:38 +0000 2018'}
-----
{'Airline': '@SpiritAirlines', 'Compound Score': 0.034161563517915314, 'Pos Score': 0.1031742671009772, 'Neg Score': 0.08340228013029315, 'Neut Score': 0.813400651465798, 'Oldest Tweet': 'Tue Oct 16 00:11:53 +0000 2018'}
-----
{'Airline': '@Virginatlantic', 'Compound Score': 0.16393761140819962, 'Pos Score': 0.12236363636363637, 'Neg Score': 0.04580926916221034, 'Neut Score': 0.8318163992869875, 'Oldest Tweet': 'Sat Oct 20 17:16:09 +0000 2018'}
-----
{'Airline': '@Delta', 'Compound Score': 0.08836128000000001, 'Pos Score': 0.1225056, 'Neg Scor

In [9]:
# Create a DataFrame using results_list and display
Total_results = pd.DataFrame(results_list).set_index("Airline").round(3)
Total_results = Total_results.sort_values("Compound Score", ascending=False)
Total_results

,Compound Score,Neg Score,Neut Score,Oldest Tweet,Pos Score
Airline,,,,,
@SouthwestAir,0.184,0.047,0.821,Mon Oct 22 14:56:28 +0000 2018,0.132
@Virginatlantic,0.164,0.046,0.832,Sat Oct 20 17:16:09 +0000 2018,0.122
@KLM,0.099,0.056,0.836,Sat Oct 20 18:59:54 +0000 2018,0.108
@AlaskaAir,0.092,0.064,0.826,Sun Oct 21 17:19:59 +0000 2018,0.110
@Delta,0.088,0.069,0.808,Mon Oct 22 20:52:41 +0000 2018,0.123
@SpiritAirlines,0.034,0.083,0.813,Tue Oct 16 00:11:53 +0000 2018,0.103
@AmericanAir,-0.003,0.083,0.831,Mon Oct 22 19:15:38 +0000 2018,0.085
@FlyFrontier,-0.013,0.084,0.841,Tue Oct 16 02:06:53 +0000 2018,0.075
